# Data Science Capstone Project
#### Destination Cities - Vacation Travel Planning

Download Dependencies Needed for Lab

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

### Destination Cities
Create a list of possible Destination Cities for a vacation

In [2]:
Vacation_Options = ["Savannah Historic District,Savannah,Georgia",
                    "Downtown Charleston,Charleston,South Carolina",
                    "Southwest Orlando,Orlando,Florida",
                    "Garden District,New Orleans,Louisiana",
                    "Fisherman's Wharf,San Francisco,California",
                    "Beacon Hill,Boston,Massachusetts",
                    "Bell Rock,Sedona,Arizona",
                    "Downtown,Key West,Florida",
                    "The Loop,Chicago,Illinois",
                    "Downtown,Houston,Texas",
                    "Downtown,Nashville,Tennessee",
                    "Union Station,Denver,Colorado",
                    "Downtown,Asheville,North Carolina",
                    "Downtown,St. Augustine,Florida"
                  ]

Loop through Vacation Options, use Nominatim geocoder to associate the proper latitude and longitude to each city, then create a dataframe from this updated list of city information, "Destination_Cities".

In [3]:
cities_latlong = []
for city in Vacation_Options:
    address = city

    geolocator = Nominatim(user_agent="destination_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    #print(''''{}',{},{}'''.format(address, latitude, longitude))
    cities_latlong.append(
        {
            'State': address.split(',', 2)[2],
            'City': address.split(',',2)[1],
            'Latitude': latitude,
            'Longitude': longitude
        }
    )
Destination_Cities = pd.DataFrame(cities_latlong)

Sort Destination_Cities dataframe by "State" and reset index

In [4]:
Destination_Cities = Destination_Cities.sort_values(by=['State']).reset_index(drop=True)

In [5]:
Destination_Cities

,State,City,Latitude,Longitude
0,Arizona,Sedona,34.831453,-111.775264
1,California,San Francisco,37.809167,-122.416599
2,Colorado,Denver,39.753630,-105.000748
3,Florida,Orlando,28.876887,-81.695584
4,Florida,Key West,26.642532,-81.862867
5,Florida,St. Augustine,29.904286,-81.319455
6,Georgia,Savannah,32.072732,-81.093158
7,Illinois,Chicago,41.881609,-87.629457
8,Louisiana,New Orleans,29.929605,-90.084388
9,Massachusetts,Boston,42.358708,-71.067829


Set the central geographical coordinates for the United States to render a folium map

In [7]:
address = 'United States'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

## Visualize Destination City List
Create a folium map of the United States with each of the destination cities

In [8]:
# create map of Destination Cities using latitude and longitude values
map_destinationcities = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for lat, lng, city, state in zip(Destination_Cities['Latitude'], Destination_Cities['Longitude'], Destination_Cities['City'], Destination_Cities['State']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_destinationcities)  
    
map_destinationcities

## Define Credentials and Call Foursquare API for City Information

In [9]:
CLIENT_ID = 'REDACTED' # your Foursquare ID
CLIENT_SECRET = 'REDACTED' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Review Foursquare information for first city on list

In [26]:
city_latitude = Destination_Cities.loc[0, 'Latitude'] # neighborhood latitude value
city_longitude = Destination_Cities.loc[0, 'Longitude'] # neighborhood longitude value

city_name = Destination_Cities.loc[0, 'City'] # neighborhood name

Set limit for top 100 venues within a radius of 8,000 meters around the specified area of the destination city

In [27]:
#The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 8000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude,
    city_longitude,  
    radius, 
    LIMIT)

Send GET request to Foursquare API

In [28]:
results = requests.get(url).json()

Define get_category_type function for Foursquare data response

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Structure data from json response into a pandas dataframe

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Little Horse Trail,Trail,34.824028,-111.775776
1,Chapel of the Holy Cross,Church,34.832122,-111.766660
2,Cathedral Rock,Scenic Lookout,34.822440,-111.789474
3,Broken Arrow Trail Head,Trail,34.845459,-111.756997
4,Son Silver West Gallery,Gift Shop,34.847054,-111.766926


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Render Vendor Information for Each Destination City
Create function to return Foursquare vendor information for each of the specified destination cities

In [32]:
def getCityVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    city_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    city_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(city_venues)

Run function for each Destination City

In [33]:
DestinationCity_venues = getCityVenues(names=Destination_Cities['City'],
                                   latitudes=Destination_Cities['Latitude'],
                                   longitudes=Destination_Cities['Longitude']
                                  )

Sedona
San Francisco
Denver
Orlando
Key West
St. Augustine
Savannah
Chicago
New Orleans
Boston
Asheville
Charleston
Nashville
Houston


Check Shape of Venue list

In [34]:
print(DestinationCity_venues.shape)
DestinationCity_venues.head()

(593, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sedona,34.831453,-111.775264,Mystic Trailhead,34.833255,-111.775751,Trail
1,Sedona,34.831453,-111.775264,Mountaintop Therapy,34.831454,-111.771963,Massage Studio
2,Sedona,34.831453,-111.775264,Sedona Fire District 9/11 Memorial,34.834600,-111.776480,Memorial Site
3,San Francisco,37.809167,-122.416599,Musée Mécanique,37.809333,-122.415952,Museum
4,San Francisco,37.809167,-122.416599,The Baked Bear,37.807447,-122.417310,Ice Cream Shop


Determine Total Venue Count for each city within set query parameters

In [36]:
DestinationCity_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Asheville,40,40,40,40,40,40
Boston,37,37,37,37,37,37
Charleston,15,15,15,15,15,15
Chicago,100,100,100,100,100,100
Denver,71,71,71,71,71,71
Houston,100,100,100,100,100,100
Key West,7,7,7,7,7,7
Nashville,53,53,53,53,53,53
New Orleans,42,42,42,42,42,42


Determine Unique Venue Categories across all Cities

In [37]:
print('There are {} uniques categories.'.format(len(DestinationCity_venues['Venue Category'].unique())))

There are 180 uniques categories.


## Analyze Each Destination City

In [38]:
# one hot encoding
DestinationCity_onehot = pd.get_dummies(DestinationCity_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DestinationCity_onehot['City'] = DestinationCity_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [DestinationCity_onehot.columns[-1]] + list(DestinationCity_onehot.columns[:-1])
DestinationCity_onehot = DestinationCity_onehot[fixed_columns]

DestinationCity_onehot.head()

,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Library,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mongolian Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Performing Arts Venue,Pet Service,Pharmacy,Piano Bar,Pizza Place,Planetarium,Platform,Playground,Plaza,Portuguese Restaurant,Pub,Public Art,Record Shop,Resort,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Sedona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,San Francisco,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,San Francisco,0,0,0,0,0,

In [39]:
DestinationCity_onehot.shape

(593, 181)

Group venues categories by City and determine the mean of the fequency of occurance

In [40]:
DestinationCity_grouped = DestinationCity_onehot.groupby('City').mean().reset_index()
DestinationCity_grouped

,City,Accessories Store,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Library,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mongolian Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Performing Arts Venue,Pet Service,Pharmacy,Piano Bar,Pizza Place,Planetarium,Platform,Playground,Plaza,Portuguese Restaurant,Pub,Public Art,Record Shop,Resort,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Asheville,0.00000,0.025000,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.00000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.025000,0.075000,0.025000,0.00000,0.000000,0.00000,0.025,0.00,0.000000,0.025,0.000000,0.000000,0.025000,0.050000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.025,0.00,0.05,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.025000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,0.100000,0.000000,0.000000,0.000000,0.000000,0.025,0.025,0.000000,0.00,0.025000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.025000,0.025000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.025000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.025000,0.00,0.000000,0.00,0.000000,0.00,0.025000,0.025000,0.025,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.025000,0.025000,0.000000,0.000000,0.025000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.00000,0.000000
1,Boston,0.00000,0.027027,0.00000,0.000000,0.00,0.000000,

In [41]:
DestinationCity_grouped.shape

(14, 181)

Display Top 5 most common venues for each City

In [85]:
num_top_venues = 5

for hood in DestinationCity_grouped['City']:
    print("----"+hood+"----")
    temp = DestinationCity_grouped[DestinationCity_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Asheville----
         venue  freq
0        Hotel  0.10
1          Bar  0.05
2         Café  0.05
3  Coffee Shop  0.05
4      Brewery  0.05


----Boston----
                venue  freq
0  Italian Restaurant  0.08
1         Pizza Place  0.08
2               Hotel  0.05
3   French Restaurant  0.05
4         Yoga Studio  0.03


----Charleston----
                 venue  freq
0                Hotel  0.25
1      Harbor / Marina  0.12
2        Boat or Ferry  0.12
3  Sporting Goods Shop  0.06
4                  Bar  0.06


----Chicago----
         venue  freq
0  Coffee Shop  0.08
1      Theater  0.06
2        Hotel  0.04
3       Bakery  0.03
4       Museum  0.02


----Denver----
                 venue  freq
0                Hotel  0.07
1           Restaurant  0.06
2         Cocktail Bar  0.04
3  American Restaurant  0.04
4   Mexican Restaurant  0.04


----Houston----
          venue  freq
0           Bar  0.10
1         Hotel  0.06
2     Nightclub  0.05
3  Cocktail Bar  0.04
4  Burger Joi

Sort Venues, created dataframe and display top 10 most common venues for each city

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
DestinationCity_venues_sorted = pd.DataFrame(columns=columns)
DestinationCity_venues_sorted['City'] = DestinationCity_grouped['City']

for ind in np.arange(DestinationCity_grouped.shape[0]):
    DestinationCity_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DestinationCity_grouped.iloc[ind, :], num_top_venues)

DestinationCity_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Asheville,Hotel,Brewery,Wine Bar,Bar,Coffee Shop,Dessert Shop,Cocktail Bar,Spa,French Restaurant,Chocolate Shop
1,Boston,Pizza Place,Italian Restaurant,Hotel,French Restaurant,Hotel Bar,Plaza,Park,Outdoor Sculpture,Other Repair Shop,Optical Shop
2,Charleston,Hotel,Boat or Ferry,Harbor / Marina,Hotel Bar,Tourist Information Center,Breakfast Spot,Boat Rental,Bar,Kitchen Supply Store,Sporting Goods Shop
3,Chicago,Coffee Shop,Theater,Hotel,Bakery,Middle Eastern Restaurant,Snack Place,Shoe Store,Concert Hall,Museum,Sandwich Place
4,Denver,Hotel,Coffee Shop,Restaurant,American Restaurant,Mexican Restaurant,Cocktail Bar,Pizza Place,Sushi Restaurant,Gym,New American Restaurant


## Cluster Destination Cities based on venues

In [44]:
# set number of clusters
kclusters = 9

DestinationCity_grouped_clustering = DestinationCity_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DestinationCity_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 6, 4, 6, 6, 1, 3, 1, 7, 2], dtype=int32)

In [60]:
# add clustering labels
DestinationCity_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DestinationCity_merged = Destination_Cities

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DestinationCity_merged = DestinationCity_merged.join(DestinationCity_venues_sorted.set_index('City'), on='City')

DestinationCity_merged # check the last columns!

,State,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arizona,Sedona,34.831453,-111.775264,0,Memorial Site,Massage Studio,Trail,Yoga Studio,Donut Shop,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,California,San Francisco,37.809167,-122.416599,5,Ice Cream Shop,Tour Provider,Historic Site,Seafood Restaurant,Bike Rental / Bike Share,Gym / Fitness Center,Food Truck,Pharmacy,Hotel,Pizza Place
2,Colorado,Denver,39.753630,-105.000748,6,Hotel,Coffee Shop,Restaurant,American Restaurant,Mexican Restaurant,Cocktail Bar,Pizza Place,Sushi Restaurant,Gym,New American Restaurant
3,Florida,Orlando,28.876887,-81.695584,2,Performing Arts Venue,Italian Restaurant,Dive Bar,Food Truck,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
4,Florida,Key West,26.642532,-81.862867,3,Theater,American Restaurant,Convenience Store,Brewery,History Museum,Science Museum,Tea Room,Dive Bar,Fast Food Restaurant,Farmers Market
5,Florida,St. Augustine,29.904286,-81.319455,5,Historic Site,Hotel,Pizza Place,History Museum,Intersection,Fast Food Restaurant,Breakfast Spot,Boutique,Museum,Fried Chicken Joint
6,Georgia,Savannah,32.072732,-81.093158,8,Plaza,Bed & Breakfast,Bookstore,American Restaurant,Museum,Bistro,Coffee Shop,Playground,Pizza Place,Breakfast Spot
7,Illinois,Chicago,41.881609,-87.629457,6,Coffee Shop,Theater,Hotel,Bakery,Middle Eastern Restaurant,Snack Place,Shoe Store,Concert Hall,Museum,Sandwich Place
8,Louisiana,New Orleans,29.929605,-90.084388,7,Public Art,Furniture / Home Store,Historic Site,Coffee Shop,Breakfast Spot,Accessories Store,Neighborhood,Bookstore,Burrito Place,Bus Stop
9,Massachusetts,Boston,42.358708,-71.067829,6,Pizza Place,Italian Restaurant,Hotel,French Restaurant,Hotel Bar,Plaza,Park,Outdoor Sculpture,Other Repair Shop,Optical Shop


#### Create Folium Map based on Clustering

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DestinationCity_merged['Latitude'], DestinationCity_merged['Longitude'], DestinationCity_merged['City'], DestinationCity_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyze Clusters

#### Cluster 1

In [62]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 0, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sedona,Memorial Site,Massage Studio,Trail,Yoga Studio,Donut Shop,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Cluster 2

In [63]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 1, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Asheville,Hotel,Brewery,Wine Bar,Bar,Coffee Shop,Dessert Shop,Cocktail Bar,Spa,French Restaurant,Chocolate Shop
12,Nashville,Hotel,Coffee Shop,Steakhouse,Cocktail Bar,Mexican Restaurant,Sushi Restaurant,Concert Hall,Music Venue,Bar,Library
13,Houston,Bar,Hotel,Nightclub,Cocktail Bar,American Restaurant,Burger Joint,New American Restaurant,Juice Bar,Grocery Store,Steakhouse


#### Cluster 3

In [64]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 2, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Orlando,Performing Arts Venue,Italian Restaurant,Dive Bar,Food Truck,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit


#### Cluster 4

In [65]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 3, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Key West,Theater,American Restaurant,Convenience Store,Brewery,History Museum,Science Museum,Tea Room,Dive Bar,Fast Food Restaurant,Farmers Market


#### Cluster 5

In [66]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 4, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Charleston,Hotel,Boat or Ferry,Harbor / Marina,Hotel Bar,Tourist Information Center,Breakfast Spot,Boat Rental,Bar,Kitchen Supply Store,Sporting Goods Shop


#### Cluster 6

In [67]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 5, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,San Francisco,Ice Cream Shop,Tour Provider,Historic Site,Seafood Restaurant,Bike Rental / Bike Share,Gym / Fitness Center,Food Truck,Pharmacy,Hotel,Pizza Place
5,St. Augustine,Historic Site,Hotel,Pizza Place,History Museum,Intersection,Fast Food Restaurant,Breakfast Spot,Boutique,Museum,Fried Chicken Joint


#### Cluster 7

In [68]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 6, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Denver,Hotel,Coffee Shop,Restaurant,American Restaurant,Mexican Restaurant,Cocktail Bar,Pizza Place,Sushi Restaurant,Gym,New American Restaurant
7,Chicago,Coffee Shop,Theater,Hotel,Bakery,Middle Eastern Restaurant,Snack Place,Shoe Store,Concert Hall,Museum,Sandwich Place
9,Boston,Pizza Place,Italian Restaurant,Hotel,French Restaurant,Hotel Bar,Plaza,Park,Outdoor Sculpture,Other Repair Shop,Optical Shop


#### Cluster 8

In [69]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 7, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,New Orleans,Public Art,Furniture / Home Store,Historic Site,Coffee Shop,Breakfast Spot,Accessories Store,Neighborhood,Bookstore,Burrito Place,Bus Stop


#### Cluster 9

In [70]:
DestinationCity_merged.loc[DestinationCity_merged['Cluster Labels'] == 8, DestinationCity_merged.columns[[1] + list(range(5, DestinationCity_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Savannah,Plaza,Bed & Breakfast,Bookstore,American Restaurant,Museum,Bistro,Coffee Shop,Playground,Pizza Place,Breakfast Spot
